#Orders Data

## Process the Orders Data
- Ingest the data into the data lakehouse - bronze orders
- Perform the data quality checks and transform the data as required - silver_orders_clean
- Apply the changes to the orders data - silver_orders

###1. Ingest the data into the data lakehouse - Bronze orders

In [0]:
USE CATALOG EcommerceDataset;

In [0]:
CREATE OR REFRESH STREAMING TABLE ecommercedataset.bronzelayer.ordersbronzetable
COMMENT 'The data is coming from the orders landing table'
TBLPROPERTIES ('quality', 'bronze')
AS 
SELECT
  *, 
  _metadata.file_path AS input_file_path, 
  CURRENT_TIMESTAMP() AS ingestion_timestamp
FROM STREAM CLOUD_FILES(
    '/Volumes/ecommercedataset/landinglayer/landingdata/',
    'json'
    )

message
The operation was successfully executed.


###2. Specifying the Expectations for the Orders table 
> Silver Orders Clean (DQ Rules)
1. Fail if customer_id is NULL
2. Fail if order_id is NULL
3. Warn if the order_status is not one of the following (Pending, Shipped, Cancelled, Completed)
4. Warn if payment_method is not one of the following (credit card, bank transfer, PayPal) 

> Silver Orders Clean (Transformations)
1. CAST order_timestamp to TIMESTAMP

In [0]:
CREATE OR REFRESH STREAMING TABLE ecommercedataset.silverlayer.orderssilverclean
(
  CONSTRAINT valid_customer_id EXPECT (customer_id is not null) ON VIOLATION FAIL UPDATE,
  CONSTRAINT valid_order_id EXPECT (order_id is not null) ON VIOLATION FAIL UPDATE,
  CONSTRAINT valid_order_status EXPECT (order_status IN ('Pending', 'Shipped', 'Cancelled', 'Completed')), 
  CONSTRAINT valid_payment_method EXPECT (payment_method IN ('Credit Card', 'Bank Transfer', 'PayPal'))
)
COMMENT 'The following table will have Data Quality Rules'
TBLPROPERTIES ('quality', 'Silver')
AS 
SELECT 
  order_id, 
  customer_id, 
  CAST(order_timestamp AS TIMESTAMP) AS order_timestamp, 
  payment_method, 
  items, 
  order_status
FROM STREAM(LIVE.ecommercedataset.bronzelayer.ordersbronzetable)

###3. Explode the items array from the order object - orderssilverclean

In [0]:
CREATE OR REFRESH STREAMING TABLE silver_orders
AS 
SELECT 
  order_id, 
  customer_id, 
  order_timestamp, 
  payment_method, 
  order_status,
  item.item_id, 
  item.name AS item_name, 
  item.price AS item_price, 
  item.quantity AS item_quantity, 
  item.category AS item_category 
FROM (
  SELECT 
    order_id, 
    customer_id, 
    order_timestamp, 
    payment_method, 
    order_status, 
    explode(items) AS item
  FROM STREAM(LIVE.orderssilverclean)
)